### 一、RFM

In [1]:
# 导入必要的库
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']

In [3]:
df_order = pd.read_csv('JD_order_data.csv')
df_user = pd.read_csv('JD_user_data.csv')

print('订单数据基本信息：')
df_order.info()
print('\n用户数据基本信息：')
df_user.info()

订单数据基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549989 entries, 0 to 549988
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   order_ID                    549989 non-null  object 
 1   user_ID                     549989 non-null  object 
 2   sku_ID                      549989 non-null  object 
 3   order_date                  549989 non-null  object 
 4   order_time                  549989 non-null  object 
 5   quantity                    549989 non-null  int64  
 6   type                        549989 non-null  int64  
 7   promise                     549989 non-null  object 
 8   original_unit_price         549989 non-null  float64
 9   final_unit_price            549989 non-null  float64
 10  direct_discount_per_unit    549989 non-null  float64
 11  quantity_discount_per_unit  549989 non-null  float64
 12  bundle_discount_per_unit    549989 non-null  float64
 13  coup

In [4]:
# 合并用户与订单数据
merged_df = pd.merge(df_order, df_user[['user_ID', 'plus']], on='user_ID', how='left')

# 转换日期格式
merged_df['order_date'] = pd.to_datetime(merged_df['order_date'])
analysis_date = merged_df['order_date'].max()  # 分析截止日期

# 剔除赠品订单(gift_item=1)
merged_df = merged_df[merged_df['gift_item'] == 0]

# 数据质量检查
print("数据基本信息：")
print(merged_df.info())
print("\n数据缺失值情况：")
print(merged_df.isnull().sum())


数据基本信息：
<class 'pandas.core.frame.DataFrame'>
Index: 455383 entries, 0 to 549988
Data columns (total 18 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   order_ID                    455383 non-null  object        
 1   user_ID                     455383 non-null  object        
 2   sku_ID                      455383 non-null  object        
 3   order_date                  455383 non-null  datetime64[ns]
 4   order_time                  455383 non-null  object        
 5   quantity                    455383 non-null  int64         
 6   type                        455383 non-null  int64         
 7   promise                     455383 non-null  object        
 8   original_unit_price         455383 non-null  float64       
 9   final_unit_price            455383 non-null  float64       
 10  direct_discount_per_unit    455383 non-null  float64       
 11  quantity_discount_per_unit  455383 n

In [5]:
merged_df

,order_ID,user_ID,sku_ID,order_date,order_time,quantity,type,promise,original_unit_price,final_unit_price,direct_discount_per_unit,quantity_discount_per_unit,bundle_discount_per_unit,coupon_discount_per_unit,gift_item,dc_ori,dc_des,plus
0,d0cf5cc6db,0abe9ef2ce,581d5b54c1,2018-03-01,2018-03-01 17:14:25.0,1,2,-,89.0,79.0,0.0,10.0,0.0,0.0,0,4,28,0
1,7444318d01,33a9e56257,067b673f2b,2018-03-01,2018-03-01 11:10:40.0,1,1,2,99.9,53.9,5.0,41.0,0.0,0.0,0,28,28,0
2,f973b01694,4ea3cf408f,623d0a582a,2018-03-01,2018-03-01 09:13:26.0,1,1,2,78.0,58.5,19.5,0.0,0.0,0.0,0,28,28,0
3,8c1cec8d4b,b87cb736cb,fc5289b139,2018-03-01,2018-03-01 21:29:50.0,1,1,2,61.0,35.0,0.0,26.0,0.0,0.0,0,4,28,0
4,d43a33c38a,4829223b6f,623d0a582a,2018-03-01,2018-03-01 19:13:37.0,1,1,1,78.0,53.0,19.0,0.0,0.0,6.0,0,3,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549982,f1a1f20e07,6208a6bceb,6cb24edcc6,2018-03-31,2018-03-31 22:06:43.0,1,2,-,59.0,39.0,0.0,0.0,0.0,20.0,0,2,2,0
549985,c9d77a7ed0,18f92434cd,7f53769d3f,2018-03-31,2018-03-31 08:55:57.0,1,1,3,118.0,55.0,63.0,0.0,0.0,0.0,0,59,2,0
549986,b9ad79338f,b5caf8a580,8dc4a01dec,2018-03-31,2018-03-31 13:31:01.0,1,1,2,78.0,78.0,0.0,0.0,0.0,0.0,0,2,2,0
549987,be3a9414b1,20ba6655f3,2dd6b818ec,2018-03-31,2018-03-31 12:51:18.0,1,2,-,189.0,78.0,111.0,0.0,0.0,0.0,0,4,28,0


In [6]:
## 二、计算RFM指标（全量数据）
rfm = merged_df.groupby(['user_ID', 'plus']).agg({
    'order_date': lambda x: (analysis_date - x.max()).days,  # Recency
    'order_ID': 'count',                                     # Frequency
    'final_unit_price': 'sum'                                # Monetary
}).reset_index()

rfm.columns = ['user_ID', 'plus', 'Recency', 'Frequency', 'Monetary']

In [7]:
## 三、调整RFM评分逻辑（按会员类型分组）
# 定义分位数计算函数
def get_quantiles(group, metric):
    return group[metric].quantile([0.2, 0.4, 0.6, 0.8]).values

# 计算分位数（不抽样，使用全量数据）
quantiles = rfm.groupby('plus').apply(lambda x: {
    'Recency': get_quantiles(x, 'Recency'),
    'Frequency': get_quantiles(x, 'Frequency'),
    'Monetary': get_quantiles(x, 'Monetary')
}).to_dict()

# 评分函数（同之前逻辑，弱化Frequency）
def rfm_scoring(x, q, metric_type):
    if metric_type == 'Recency':
        if x <= q[0]: return 5
        elif x <= q[1]: return 4
        elif x <= q[2]: return 3
        elif x <= q[3]: return 2
        else: return 1
    elif metric_type == 'Frequency':
        return 5 if x >= 2 else 1  # 简化评分
    else:
        if x > q[3]: return 5
        elif x > q[2]: return 4
        elif x > q[1]: return 3
        elif x > q[0]: return 2
        else: return 1

# 应用评分函数
rfm['R'] = rfm.apply(lambda row: rfm_scoring(row['Recency'], quantiles[row['plus']]['Recency'], 'Recency'), axis=1)
rfm['F'] = rfm.apply(lambda row: rfm_scoring(row['Frequency'], quantiles[row['plus']]['Frequency'], 'Frequency'), axis=1)
rfm['M'] = rfm.apply(lambda row: rfm_scoring(row['Monetary'], quantiles[row['plus']]['Monetary'], 'Monetary'), axis=1)


C:\Users\zoey_\AppData\Local\Temp\ipykernel_37060\1739559347.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  quantiles = rfm.groupby('plus').apply(lambda x: {


In [8]:
print("F评分分布：")
print(rfm['F'].value_counts(normalize=True))

F评分分布：
F
1    0.902666
5    0.097334
Name: proportion, dtype: float64


In [9]:
## 四、客户分群（基于R和M）
def segment_customers(row):
    r, m = row['R'], row['M']
    if r >= 4 and m >= 5:          # 高价值客户：R前20%活跃 + M前20%消费
        return '高价值客户'
    elif r >= 4 and m >= 3:       # 潜力客户：R前20%活跃 + M前40%-60%消费
        return '潜力客户'
    elif r < 4 and m >= 4:        # 高消费沉睡客户：R后80%不活跃 + M前20%-40%消费
        return '高消费沉睡客户'
    else:
        return '低价值客户'

rfm['Segment'] = rfm.apply(segment_customers, axis=1)


In [10]:
## 五、结果分析与可视化
# 各分群基础统计
segment_stats = rfm.groupby('Segment')[['Recency', 'Frequency', 'Monetary', 'R', 'F', 'M']].agg(['mean', 'count'])
print("\n=== 各分群RFM指标统计 ===")
print(segment_stats)

# 计算每个分群的RFM分数分布
segment_rfm_scores = rfm.groupby('Segment').agg({
    'R': ['mean', 'std'],
    'F': ['mean', 'std'],
    'M': ['mean', 'std']
}).round(2)

print("\n各分群的RFM分数分布：")
print(segment_rfm_scores)

# 客户占比
segment_proportion = rfm['Segment'].value_counts(normalize=True).reset_index()
print("\n=== 各分群客户占比 ===")
print(segment_proportion)

# PLUS会员比例
plus_ratio = rfm.groupby('Segment')['plus'].mean().reset_index()
print("\n=== 各分群PLUS会员比例 ===")
print(plus_ratio)

# 计算每个分群中plus会员和非plus会员的占比
segment_plus_dist = rfm.groupby(['Segment', 'plus']).size().unstack(fill_value=0)
segment_plus_dist['plus_0_占比'] = segment_plus_dist[0] / segment_plus_dist[0].sum() * 100
segment_plus_dist['plus_1_占比'] = segment_plus_dist[1] / segment_plus_dist[1].sum() * 100

print("\n各分群中plus会员和非plus会员的分布：")
print(segment_plus_dist.round(2))


=== 各分群RFM指标统计 ===
           Recency         Frequency            Monetary                 R  \
              mean   count      mean   count        mean   count      mean   
Segment                                                                      
低价值客户    17.120507  208635  1.025082  208635   48.824746  208635  2.772675   
潜力客户      5.009546   72281  1.105616   72281   89.644803   72281  4.481966   
高价值客户     4.684274   34815  1.483987   34815  236.667978   34815  4.533276   
高消费沉睡客户  21.895232   88157  1.247037   88157  164.238152   88157  2.104461   

                        F                 M          
          count      mean   count      mean   count  
Segment                                              
低价值客户    208635  1.095900  208635  1.797167  208635  
潜力客户      72281  1.389093   72281  3.458364   72281  
高价值客户     34815  2.113773   34815  5.000000   34815  
高消费沉睡客户   88157  1.797895   88157  4.516907   88157  

各分群的RFM分数分布：
            R           F           M    

In [11]:
# 各分群内会员与非会员的 Monetary 均值对比
member_stats = rfm.groupby(['Segment', 'plus'])['Monetary'].agg(['mean', 'count']).reset_index()
member_stats.columns = ['Segment', 'plus', 'Avg_Monetary', 'Count']
print(member_stats)

   Segment  plus  Avg_Monetary   Count
0    低价值客户     0     48.863093  171746
1    低价值客户     1     48.646210   36889
2     潜力客户     0     85.939225   58878
3     潜力客户     1    105.923028   13403
4    高价值客户     0    228.437802   28452
5    高价值客户     1    273.469011    6363
6  高消费沉睡客户     0    157.259354   72103
7  高消费沉睡客户     1    195.581885   16054


In [12]:
total_member_spend = rfm[rfm['plus'] == 1]['Monetary'].sum()
high_value_member_spend = rfm[(rfm['Segment'] == '高价值客户') & (rfm['plus'] == 1)]['Monetary'].sum()
print(f"高价值会员消费占比：{high_value_member_spend / total_member_spend * 100:.2f}%")

高价值会员消费占比：21.50%
